## Chapter 7: Decks

Icepool also has some support for decks.

### Creation

A `Deck` is constructed similarly to a `Die`; in this case, quantities represent the number of each type of card in the deck rather than weights.

For example, here's a deck consisting of each of the numbers from 1-13, with 4 cards per number:

In [1]:
import piplite
await piplite.install("icepool")

from icepool import Deck, Die

print(Deck(range(1, 14), times=4))

Deck with denominator 52

| Outcome | Quantity | Probability |
|--------:|---------:|------------:|
|       1 |        4 |   7.692308% |
|       2 |        4 |   7.692308% |
|       3 |        4 |   7.692308% |
|       4 |        4 |   7.692308% |
|       5 |        4 |   7.692308% |
|       6 |        4 |   7.692308% |
|       7 |        4 |   7.692308% |
|       8 |        4 |   7.692308% |
|       9 |        4 |   7.692308% |
|      10 |        4 |   7.692308% |
|      11 |        4 |   7.692308% |
|      12 |        4 |   7.692308% |
|      13 |        4 |   7.692308% |




Note that nested collections are treated differently compared to the `Die` constructor.

With a `Die`, the assigned quantities are interpreted as weights. Each sub-collection is treated as a die, and the weight assigned to it is effectively shared between all of its outcomes. The overall denominator increases to be able to represent all the sub-dice if necessary.

With a `Deck`, the assigned quantities are interpreted as duplicates. Each outcome is considered to be a card, and is duplicated the specified number of times.

For example, here's the same `Deck` with two extra 14s, compared to a `Die` constructed with the same arguments:

In [2]:
# Take a deck of 1-13, duplicate it 4 times, then throw in two 14s.
deck = Deck({Deck(range(1, 14)): 4, 14: 2})
print(deck)

# Roll 1d6:
# On 1-4: the result is 1d13.
# On 5-6: the result is 14.
print(Die({Die(range(1, 14)): 4, 14: 2}))

Deck with denominator 54

| Outcome | Quantity | Probability |
|--------:|---------:|------------:|
|       1 |        4 |   7.407407% |
|       2 |        4 |   7.407407% |
|       3 |        4 |   7.407407% |
|       4 |        4 |   7.407407% |
|       5 |        4 |   7.407407% |
|       6 |        4 |   7.407407% |
|       7 |        4 |   7.407407% |
|       8 |        4 |   7.407407% |
|       9 |        4 |   7.407407% |
|      10 |        4 |   7.407407% |
|      11 |        4 |   7.407407% |
|      12 |        4 |   7.407407% |
|      13 |        4 |   7.407407% |
|      14 |        2 |   3.703704% |


Die with denominator 78

| Outcome | Quantity | Probability |
|--------:|---------:|------------:|
|       1 |        4 |   5.128205% |
|       2 |        4 |   5.128205% |
|       3 |        4 |   5.128205% |
|       4 |        4 |   5.128205% |
|       5 |        4 |   5.128205% |
|       6 |        4 |   5.128205% |
|       7 |        4 |   5.128205% |
|       8 |        4 |

### Deals

Since cards are drawn without replacement, they cannot be considered independent of each other. Thus, for a deck to be useful, we need to specify how many cards to deal from it. This is done using the `deal` method. This produces a `Deal` object. In turn, this is a subclass of `MultisetGenerator`---making a `Deal` analogous to a `Pool`.

In fact, `Deal`s have all the same built-in evaluators as `Pool`s, and can be used with custom `MultisetEvaluator` just as well as a `Pool`.

For example, here's the sum of two cards:

In [3]:
deal = deck.deal(2)

print(deal.sum())

Die with denominator 1431

| Outcome | Quantity | Probability |
|--------:|---------:|------------:|
|       2 |        6 |   0.419287% |
|       3 |       16 |   1.118099% |
|       4 |       22 |   1.537386% |
|       5 |       32 |   2.236198% |
|       6 |       38 |   2.655486% |
|       7 |       48 |   3.354298% |
|       8 |       54 |   3.773585% |
|       9 |       64 |   4.472397% |
|      10 |       70 |   4.891684% |
|      11 |       80 |   5.590496% |
|      12 |       86 |   6.009783% |
|      13 |       96 |   6.708595% |
|      14 |      102 |   7.127883% |
|      15 |      104 |   7.267645% |
|      16 |       94 |   6.568833% |
|      17 |       88 |   6.149546% |
|      18 |       78 |   5.450734% |
|      19 |       72 |   5.031447% |
|      20 |       62 |   4.332635% |
|      21 |       56 |   3.913347% |
|      22 |       46 |   3.214535% |
|      23 |       40 |   2.795248% |
|      24 |       30 |   2.096436% |
|      25 |       24 |   1.677149% |
|      26 |

You can also deal multiple hands from the same deck (without replacing between them). This causes each hand to take up multiple "slots" within an `MultisetEvaluator`.

As an example, let's take exactly the same *RISK*-like evaluator from the last chapter and deal 3 cards to player A and 2 cards to player B:

In [4]:
from icepool import MultisetEvaluator, Order

class RiskEvaluator(MultisetEvaluator):
    def next_state(self, state, outcome, a, b):
        if state is None:
            score_a, score_b, advantage = 0, 0, 0
        else:
            score_a, score_b, advantage = state
        # Advantage is the number of unpaired dice that rolled a previous (higher) number.
        # If positive, it favors side A, otherwise it favors side B.
        # We pair them off with newly-rolled dice of the disadvantaged side.
        if advantage > 0:
            score_a += min(b, advantage)
        elif advantage < 0:
            score_b += min(a, -advantage)
        advantage += a - b
        return score_a, score_b, advantage
    
    def final_outcome(self, final_state):
        # Take only the scores.
        return final_state[:2]
    
    def order(self, *_):
        # See outcomes in descending order.
        return Order.Descending

risk_evaluator = RiskEvaluator()

print(risk_evaluator(deck.deal(3, 2)))

Die with denominator 31625100

| Outcome[0] | Outcome[1] | Quantity | Probability |
|-----------:|-----------:|---------:|------------:|
|          0 |          0 |   185640 |   0.587002% |
|          0 |          1 |  1517464 |   4.798290% |
|          0 |          2 |  5510596 |  17.424754% |
|          1 |          0 |  2337868 |   7.392445% |
|          1 |          1 |  7519408 |  23.776709% |
|          2 |          0 | 14554124 |  46.020800% |




Here the `Deal` takes up both slots of the evaluator. If you wanted to, you could even mix `Deck`s and `Pool`s in the arguments to an evaluation.